In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer # tfidf matrix
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import re
from datetime import time
from datetime import datetime
from datetime import timedelta
from geopy.geocoders import Nominatim
from IPython.core.display import HTML


In [120]:
shop_db = pd.DataFrame({
    'shop ID': [        
        '111222a',
        '111333b',
        '111444a',
        '111555b', 
        '555111b', 
    ], 
    'shop name': [
        'Farmàcia Santa Maria',
        "Farmàcia del Monestir",
        "Farmàcia Beringues", 
        'Farmàcia Sant Cugat TORREBLANCA', 
        'Farmàcia Francesc Macià'
    ],
    'adress' :[
        'Carrer de Santa Maria', 
        'Carrer de Santiago Rusiñol',
        'Av. Lluís Companys i Jover', 
        'Rambla del Celler', 
        'Passeig de Francesc Macià'
        
    ],
    'number' :[
        27, 
        38, 
        46, 
        97, 
        34
    ],
    'zip code': [
        '08172',
        '08172',
        '08172',
        '08172',
        '08172'

    ],
    'shop lat': [
        42.391,
        12.381,
        90.595,
        51.102,
        44.004,
    
    ],
    'shop lon':[
        8.1943,
        2.583,
        19.13,
        27.523,
        1.13,
    
    ],
    'shop ratting': [
        4.7,
        4.3,
        3.8,
        4.2,
        3.7
    ],
    'opening time' : [
        pd.to_datetime("09:00", format="%H:%M"), 
        pd.to_datetime("09:30", format="%H:%M"), 
        pd.to_datetime("10:00", format="%H:%M"), 
        pd.to_datetime("07:30", format="%H:%M"), 
        pd.to_datetime("08:00", format="%H:%M")
        
    ], 
    'closing time' : [
        pd.to_datetime("21:00", format="%H:%M"), 
        pd.to_datetime("20:00", format="%H:%M"), 
        pd.to_datetime("20:00", format="%H:%M"), 
        pd.to_datetime("20:30", format="%H:%M"), 
        pd.to_datetime("21:00", format="%H:%M")
    ]

}, columns=['shop ID', 'shop name','adress','number', 'zip code', 'shop lat', 'shop lon', 'shop ratting', 'opening time', 
           'closing time'], 
)

In [121]:
product_db = pd.DataFrame({
    'shop ID': [        
        '111222a',
        '111222a',
        '111222a',
        '111222a',
        
        '111333b',
        '111333b',
        '111333b',
        '111333b',
        
        '111444a',
        '111444a',
        '111444a',
        
        '111555b', 
        '111555b', 
        
        '555111b',
        '555111b',
        '555111b',
    ], 
    'product ID' : [
        'm123456',
        'm314594', 
        'm847265', 
        'm957365',
        
        'm123456',
        'm314544', 
        'm811285', 
        'm900365',
        
        'm133456',
        'm388594', 
        'm899265', 
        
        'm123456',
        'm314994', 
        
        'm123236',
        'm314984', 
        'm847455' 

    ], 
    'product name': [
        'lipitor(atorvastatina)', 
        'Nexium (esomeprazol)',
        'Singulair (montelukast)', 
        'Abilify (aripiprazol)', 
        
        'lipitor(atorvastatina)', 
        'Nexium (esomeprazol)',
        'Singulair (montelukast)', 
        'Abilify (aripiprazol)', 
        
        'lipitor(atorvastatina)', 
        'Nexium (esomeprazol)',
        'Singulair (montelukast)', 

        'lipitor(atorvastatina)', 
        'Nexium (esomeprazol)',

        'lipitor(atorvastatina)', 
        'Nexium (esomeprazol)',
        'Singulair (montelukast)'
        
    ],
    'product description' : [
        'Lipitor (atorvastatin) is a medication commonly used to treat high cholesterol and prevent heart disease. ', 
        'Nexium (esomeprazole) is a medication commonly used to treat gastroesophageal reflux disease (GERD) and other conditions involving excessive stomach acid such as Zollinger-Ellison syndrome.',  
        'Singulair (montelukast) is a medication commonly used to treat asthma and allergies such as hay fever (seasonal allergic rhinitis) and perennial allergic rhinitis (year-round allergies).',
        'Abilify (aripiprazole) is a medication commonly used to treat mental health conditions such as schizophrenia, bipolar disorder, and major depressive disorder.',
        
        'Lipitor (atorvastatin) is a medication commonly used to treat high cholesterol and prevent heart disease. ', 
        'Nexium (esomeprazole) is a medication commonly used to treat gastroesophageal reflux disease (GERD) and other conditions involving excessive stomach acid such as Zollinger-Ellison syndrome.',  
        'Singulair (montelukast) is a medication commonly used to treat asthma and allergies such as hay fever (seasonal allergic rhinitis) and perennial allergic rhinitis (year-round allergies).',
        'Abilify (aripiprazole) is a medication commonly used to treat mental health conditions such as schizophrenia, bipolar disorder, and major depressive disorder.',
    
        'Lipitor (atorvastatin) is a medication commonly used to treat high cholesterol and prevent heart disease. ', 
        'Nexium (esomeprazole) is a medication commonly used to treat gastroesophageal reflux disease (GERD) and other conditions involving excessive stomach acid such as Zollinger-Ellison syndrome.',  
        'Singulair (montelukast) is a medication commonly used to treat asthma and allergies such as hay fever (seasonal allergic rhinitis) and perennial allergic rhinitis (year-round allergies).',
        
        'Lipitor (atorvastatin) is a medication commonly used to treat high cholesterol and prevent heart disease. ', 
        'Nexium (esomeprazole) is a medication commonly used to treat gastroesophageal reflux disease (GERD) and other conditions involving excessive stomach acid such as Zollinger-Ellison syndrome.',  
  
        'Lipitor (atorvastatin) is a medication commonly used to treat high cholesterol and prevent heart disease. ', 
        'Nexium (esomeprazole) is a medication commonly used to treat gastroesophageal reflux disease (GERD) and other conditions involving excessive stomach acid such as Zollinger-Ellison syndrome.',  
        'Singulair (montelukast) is a medication commonly used to treat asthma and allergies such as hay fever (seasonal allergic rhinitis) and perennial allergic rhinitis (year-round allergies).'  
        
    ], 
    'price in euros' : [
        45.67,
        243.54,
        534.87, 
        132.45, 
        
        45.42,
        241.43,
        531.24, 
        147.89, 
        
        55.23,
        281.54,
        571.76, 
        
        57.75,
        281.47,
        
        52.47,
        221.44,
        531.45

        
    ], 
    'product stock' : [
        4, 
        0, 
        523,
        54, 
        
        54, 
        43, 
        0, 
        45, 
        
        54, 
        57, 
        0, 
        
        56, 
        6, 
        
        65, 
        56, 
        7, 
        
    ]

}, columns=['shop ID', 'product ID','product name','product description','price in euros', 'product stock'], 
)

product_db['photo'] = [
        "photos/Lipitor.jpg",
        "photos/nexium.jpg",
        "photos/singulair.jpg", 
        "photos/abilify.jpg", 
        
        "photos/Lipitor.jpg",
        "photos/nexium.jpg",
        "photos/singulair.jpg", 
        "photos/abilify.jpg", 
        
        "photos/Lipitor.jpg",
        "photos/nexium.jpg",
        "photos/singulair.jpg", 

        "photos/Lipitor.jpg",
        "photos/nexium.jpg",

        "photos/Lipitor.jpg",
        "photos/nexium.jpg",
        "photos/singulair.jpg",  
    ]
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

product_db.to_html(escape=False, formatters=dict(photo=path_to_image_html))
# per mirar imagtes
HTML(product_db.to_html(escape=False,formatters=dict(photo=path_to_image_html)))
# dataFrame en sí -> product_df.head()

,shop ID,product ID,product name,product description,price in euros,product stock,photo
0,111222a,m123456,lipitor(atorvastatina),Lipitor (atorvastatin) is a medication commonly used to treat high cholesterol and prevent heart disease.,45.67,4,
1,111222a,m314594,Nexium (esomeprazol),Nexium (esomeprazole) is a medication commonly used to treat gastroesophageal reflux disease (GERD) and other conditions involving excessive stomach acid such as Zollinger-Ellison syndrome.,243.54,0,
2,111222a,m847265,Singulair (montelukast),Singulair (montelukast) is a medication commonly used to treat asthma and allergies such as hay fever (seasonal allergic rhinitis) and perennial allergic rhinitis (year-round allergies).,534.87,523,
3,111222a,m957365,Abilify (aripiprazol),"Abilify (aripiprazole) is a medication commonly used to treat mental health conditions such as schizophrenia, bipolar disorder, and major depressive disorder.",132.45,54,
4,111333b,m123456,lipitor(atorvastatina),Lipitor (atorvastatin) is a medication commonly used to treat high cholesterol and prevent heart disease.,45.42,54,
5,111333b,m314544,Nexium (esomeprazol),Nexium (esomeprazole) is a medication commonly used to treat gastroesophageal reflux disease (GERD) and other conditions involving excessive stomach acid such as Zollinger-Ellison syndrome.,241.43,43,
6,111333b,m811285,Singulair (montelukast),Singulair (montelukast) is a medication commonly used to treat asthma and allergies such as hay fever (seasonal allergic rhinitis) and perennial allergic rhinitis (year-round allergies).,531.24,0,
7,111333b,m900365,Abilify (aripiprazol),"Abilify (aripiprazole) is a medication commonly used to treat mental health conditions such as schizophrenia, bipolar disorder, and major depressive disorder.",147.89,45,
8,111444a,m133456,lipitor(atorvastatina),Lipitor (atorvastatin) is a medication commonly used to treat high cholesterol and prevent heart disease.,55.23,54,
9,111444a,m388594,Nexium (esomeprazol),Nexium (esomeprazole) is a medication commonly used to treat gastroesophageal reflux disease (GERD) and other conditions involving excessive stomach acid such as Zollinger-Ellison syndrome.,281.54,57,


In [122]:
shop_db

,shop ID,shop name,adress,number,zip code,shop lat,shop lon,shop ratting,opening time,closing time
0,111222a,Farmàcia Santa Maria,Carrer de Santa Maria,27,08172,42.391,8.1943,4.7,1900-01-01 09:00:00,1900-01-01 21:00:00
1,111333b,Farmàcia del Monestir,Carrer de Santiago Rusiñol,38,08172,12.381,2.5830,4.3,1900-01-01 09:30:00,1900-01-01 20:00:00
2,111444a,Farmàcia Beringues,Av. Lluís Companys i Jover,46,08172,90.595,19.1300,3.8,1900-01-01 10:00:00,1900-01-01 20:00:00
3,111555b,Farmàcia Sant Cugat TORREBLANCA,Rambla del Celler,97,08172,51.102,27.5230,4.2,1900-01-01 07:30:00,1900-01-01 20:30:00
4,555111b,Farmàcia Francesc Macià,Passeig de Francesc Macià,34,08172,44.004,1.1300,3.7,1900-01-01 08:00:00,1900-01-01 21:00:00


In [123]:
def search(title, names):
    vectorizer = TfidfVectorizer(analyzer = 'char_wb') 
    tfidf = vectorizer.fit_transform(names)

    
    title = clean_title(title)
    query_vec = vectorizer.transform([title])
    # we compare how similar is the title entered with all titles
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    # to select the five more similar titles
    indices = np.argpartition(similarity, -1)[-1:]
    print(indices)
    return names[indices]

In [124]:
def add_product(attr_list, product_db):
    '''
    Function used by the owner for adding a new product to the DataBase.
    attr_list: [shop ID, product name, product description, price in euros, product stock]
    '''
    global ID_product_count
    
    attr_list.insert(1, ID_product_count )
    
    product_db.loc[product_db.shape[0]] = attr_list
    
    ID_product_count += 1
    
    return product_db
    
    
def add_shop(attr_list, shop_db):
    '''
    Function used by the owner for adding a new shop to the DataBase.
    attr_list: [shop name, adress, number, zip code, opening time hour, opening time minute, closing time hour,
                closing time minute] 
    '''
    global ID_shop_count
    
    geolocator = Nominatim(user_agent="MyApp")
    shop_name = attr_list[0]
    adress, number, zipcode = attr_list[1], attr_list[2], attr_list[3]
    loc = geolocator.geocode(adress+', '+str(number)+', '+str(zipcode))
    shop_lat = loc.latitude
    shop_lon = loc.longitude
    
    open_h, open_m = attr_list[4], attr_list[5]
    close_h, close_m = attr_list[6], attr_list[7]
    
    op_time = time(hour=open_h, minute=open_m)
    cl_time = time(hour=close_h, minute=close_m)
    
    add = [ID_shop_count, shop_name, adress, number, zipcode, shop_lat, shop_lon, 1, op_time, cl_time ]
    shop_db.loc[shop_db.shape[0]] = add
    
    ID_shop_count += 1
    
    return shop_db

In [130]:
def update_stock_down(product_db, productID, n_products):
    product_stock = product_db[product_db['product ID'] == productID]['product stock'].iloc[0]
    
    if product_stock - n_products < 0:
        return None
    
    product_db[product_db['product ID'] == productID]['product stock'] -= n_products
    
    return product_db


def update_stock_up(product_db, productID, n_products):
    product_db[product_db['product ID'] == productID]['product stock'] +=  n_products
    
    return product_db

In [ ]:
def update_price(product_db, productID, new_price):
    product_db[product_db['product ID'] == productID]['price in euros'] = new_price

In [125]:
# Hyperparameters
open_threshold = 3 # If there are less than open_threshold shops opened, show also closed shops
stock_threshold = 3 # If there are less than stock_threshold shops with the product on stock, show also shops without the
                    # product on stock
ID_product_count = 0 # ID Count. Adds by one each time a product is added.
ID_shop_count = 0

In [126]:
def find_features(query, product_db, shop_db, user_db, city_db, city, sorting_criteria = 'price', user_coords=None):
    '''
    Function used by the user to look for the nearest products.
    Returns a list of the products asked for, ordered by distance.
    query: string containing the user's query.
    sorting_criteria: 'price','location', 'ratting'
    user_coords: latitude, longitude
    '''
    
    item_set = set(product_db['product name'].to_numpy())
    item_query = search(query, item_set)
    
    # Only Show items queried by the user
    items_db_query = product_db[product_db['product name'] == product_query]
    
    
    # Show only items from zip code
    zip_code = city_db[city_db['city name'] == city]['zip code'].iloc[0] # Get city zip code
    
    items_db_query = pd.merge(items_db_query, shop_db, how='left', on='shop_ID')
    items_db_query = items_db_query[items_db_query['zip code']==zip_code]
    
    
    # Si hi ha menys de open_threshold tendes obertes, mostra també les que estan tancades 
    # Si hi ha més de open_threshold tendes no es mostra
    now = datetime.now()
    now_sec = now.hour*60*60+now.minute*60
    
    opened = items_db_query['opening time'].hour*60*60 + items_db_query['opening time'].minute*60 < now_sec
    not_closed = items_db_query['closing time'].hour*60*60 + items_db_query['closing time'].minute*60 > now_sec
    
    if sum(opened & not_closed) > open_threshold: # Hi ha menys de 3 tendes obertes
        items_db_query = items_db_query[opened & not_closed] # Show only opened shops
        
        
    # Si hi ha menys de stock_threshold tendes amb el producte a stock, mostra també el que no el tenen en stock
    # Si hi ha més de stock_threshold tendes no es mostra
    on_stock = items_db_query['product stock'] > 0
    
    if sum(on_stock) > stock_threshold:
        items_db_query = items_db_query[on_stock]
        
    else:
        items_db_query['on stock'] = on_stock
        items_db_query.sort_values('on stock', ascending=False, inplace=True)
        
    # Sorting criterion
    if sorting_criteria == 'location' and user_coords != None:
        items_bd_query['distances'] = list(map( calc_distance, user_coords[0], user_coords[1], items_db_query['shop lat']
                                                     , items_db_query['shop lon'] ))

        items_db_query.sort_values('distances', ascending=True, inplace=True)
        
    elif sorting_criteria == 'price':
        items_db_query.sort_values('price in euros', ascending=True, inplace=True)
    
    elif sorting_criteria == 'ratting':
        items_db_query.sort_values('shop ratting', ascending=False, inplace=True)
        
    
    return items_db_query

In [127]:
def calc_distance(lat1, lon1, lat2, lon2):
    '''
    Calculates distance between two points.
    1 = user_coords
    2 = shop_coords
    '''
    return math.acos(math.sin(lat1)*math.sin(lat2)+math.cos(lat1)*math.cos(lat2)*math.cos(lon2-lon1))*6371

In [73]:
a = pd.DataFrame([['asd',2,3, 'carrer sunyer, la Ràpita'],[4,5,6, 'calle santa maria 27']],columns=['Hola','xUser','yUser', 'adress'])

In [128]:
new_shops = [
    ['shop name', 
     'adress', 'number', 
     'zip_code', 
     'opening Hour', 'Opening Minute', 
     'Closing Hour', 'Closing Minute'], 
    ['Ferreteria Rambla Celler', 
    'Carrer de Sant Ramon', 5, 
    '08172', 
    9, 30, 
    21, 0], # id_shop -> 321123a
    ['Mercasol Serveis i Reparacions, S.L. ', 
    'Carrer del Roselló', 20,
    '08172', 
    7, 30, 
    20, 30], # id_shop -> 333333a
    ['Ferreteria Suministres Ros', 
    'Carrer del Dos de Maig', 12, 
    '08172', 
    8,0, 
    20, 45] # id_shop -> 311113a
]

new_products = [
    ['shop_id', 'name', 
     'description', 
     'price','stock', 
     'photo'], 
    ['321123a', 'Hammer Drill', 
     ' power tool designed to perform drilling tasks on hard materials such as masonry, concrete or metal', 
     25.00, 45, 
     "photos/taladrop.jpg"
    ],
    ['321123a', 'Electric angle grinder',
    'electric tool used for cutting, sanding, polishing and grinding different materials', 
    24.04, 0,
     "photos/anglegrinder.jpg"
    ],
    ['321123a', 'Jigsaw' , 
    'A jigsaw is a type of power tool that is primarily used for cutting curves and intricate shapes in wood, metal, plastic, and other materials', 
    22.97, 67, 
    "photos/jigsaw.jpg"],
    ['321123a', 'Silicone mini gun', 
    'A silicone mini gun is a tool used for dispensing silicone sealant or adhesive.', 
    4.62, 2, 
    "photos/gun.jpg"], 
    
    ['333333a', 'Hammer Drill', 
     ' power tool designed to perform drilling tasks on hard materials such as masonry, concrete or metal', 
     20.76, 5, 
     "photos/taladrop.jpg"
    ],
    ['333333a', 'Electric angle grinder',
    'electric tool used for cutting, sanding, polishing and grinding different materials', 
    24.40, 30,
     "photos/anglegrinder.jpg"
    ],
    ['333333a', 'Jigsaw' , 
    'A jigsaw is a type of power tool that is primarily used for cutting curves and intricate shapes in wood, metal, plastic, and other materials', 
    22.76, 79, 
    "photos/jigsaw.jpg"],

    ['311113a', 'Electric angle grinder',
    'electric tool used for cutting, sanding, polishing and grinding different materials', 
    32.48, 30,
     "photos/anglegrinder.jpg"
    ],
    ['311113a', 'Jigsaw' , 
    'A jigsaw is a type of power tool that is primarily used for cutting curves and intricate shapes in wood, metal, plastic, and other materials', 
    31.60, 6, 
    "photos/jigsaw.jpg"],
    ['311113a', 'Silicone mini gun', 
    'A silicone mini gun is a tool used for dispensing silicone sealant or adhesive.', 
    5.62, 29, 
    "photos/gun.jpg"], 
]